In [4]:
from torchsummary import summary
import torch
from collections import deque
from unityagents import UnityEnvironment
import numpy as np
from dqn_agent import Agent
from model import QNetwork

# Question for banana navigation

Several questions  
Q1: why there are two Q value in this project Q_expected, Q_targets and why there are two model qnetwork_target, qnetwork_local?
 - Q_targets: say we take a single move in current state and see what reward(Qvalue) we can get, thus, reward+Q(s',a') become the output(target) that we want Q(s,a) to be, and this is the defination of qnetwork_target.  
  `Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))`  # done=0 or 1
  
 - Q_expected: Q value from the model:  
  `Q_expected = self.qnetwork_local(states).gather(1, actions)`
  
 - qnetwork_local:  
     the weights are calculated by the current(local) batch

  
 - qnetwork_target:  
     designed to be a global model that can give a ideal Q value in each state-action pair regardless of the variation of   states. 
 
Explaination: why we need to network qnetwork_local and qnetwork_target?  
  > At every step of training, the Q-network’s values shift, and if we are using a constantly shifting set of values to adjust our network values, then the value estimations can easily spiral out of control. The network can become destabilized by falling into feedback loops between the target and (local batch)estimated Q-values.  
   **We want the qnetwork_target can handle every kinds of state and give ideal reward like the qnetwork_local dose??** so we use the mes_loss(mean square loss) to calculate the mean square difference between  Q_expected and Q_targets.  
   the weights in qnetwork_target are then slowly update by the function: `θ_target = τ*θ_local + (1-τ)*θ_target`

----
Q2:can i regard the Q_value: **Q(St,At)** as the reward value?   
Reply: No, Q_value is a quantitative measure of an action in a state.  
**Me: Maybe my explaination of loss function (see above) are not correct. As i don't understand why reward valuse is only used in Q_targets, not in Q_expected. How can the weights in Q_expected being updated without knowing the supervised info(reward value)?**





# Deep Q-learning Network for banana navigation

***In this project, the goal of the agent is to collect yellow banana and avoid blue banana.***

Given a state as a vector in length=37(containing the agent's velocity, along with ray-based perception of objects around agent's forward direction), the agent are expected to select the action that bring biggest **current** value.  
Although the state is discrete and not very large(37), sampling the states will influence the performance. Thus, DQN is sellected as the method for this project.

**how dose DQN work?**  
- feed forward  
> taking a state value as input, the feed-forward QNetwork(in this project, a three layers' fully connected network) output the values on all actions. the action is selected with epsilon-greedy method. With probability $\epsilon$ the agent randomly selected an action, and with prob (1- $\epsilon$), the agent select the action with maxium output value. 

- back propagation
> loss is calculated by the mean square difference of Q_expected, Q_targets. Q_expected is the network's value of the selected action while Q_targets is the reward of current action plus the Q_targets_next with discount rate $\gamma$. I haven't totally understand why we doing this..., so i left some place here.

**selection of the hyprer parameters**
 - $\epsilon$(epsilon greedy action discount factor):  
With prob=$\epsilon$,the agent randomly selects an action.With prob=(1-$\epsilon$), the agent selects the action with max $\epsilon$ is decreaing with the increase of epoch, which means the agent tend to be more confinent of the action with the maxium action value (Q value)  
$\epsilon$ = $\epsilon$ * 0.99 (0.99 is the multiplicative factor for decreasing the epsilon value)
 - $\gamma$(discount factor, the influence of next state's Q value)  
 used in Q_targets, `Q_targets = rewards + (gamma * Q_targets_next * (1 - dones)) `
 - LR: learning rate for updating the weights in model
 - n_episodes: how many rounds are designed for agent to play the game
 - Adam optimzer: 




# Visualize the layer of the model and the curve while training

In [13]:
state_size = 37
action_size = 4
seed = 0
model = QNetwork(state_size, action_size, seed)
summary(model, input_size=(37,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]           4,864
            Linear-2                   [-1, 64]           8,256
            Linear-3                    [-1, 4]             260
Total params: 13,380
Trainable params: 13,380
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (MB): 0.05
----------------------------------------------------------------


## here we plot the episode-reward curve durning training

![Aaron Swartz](https://raw.githubusercontent.com/YCyuchen/RL_Navigation_Unity/master/score-episode-curve.png)

# Further work
in this work, the agent achieve average 14 yellow bananas in each episode. This number can be expected to be higher.  
For achieving this goal, severl methods are proposed here:
- replace the model(3 fully connected layers) with 1d conv layer. This may help the agent to learn the spatial info from the environment and make better decision.
- Improve the structure of DQN, like using Dueling DQN 、 Double Q-Learning、 or other [great off-policy RL algorithm](Demystifying Rainbow Baseline)